In [24]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import trl
import torch
import transformers

import pandas as pd
from random import randint
from datasets import Dataset, load_dataset

from trl import SFTTrainer
from peft import LoraConfig

import wandb
from transformers import (AutoTokenizer,
                          AutoModelForCausalLM,
                          TrainingArguments,
                        )

from huggingface_hub import login

import os
import json 
from datetime import datetime
from openai import OpenAI

# 환경변수 로드
from dotenv import load_dotenv
load_dotenv("./credit-env")

In [25]:
# 제로샷 테스트를 하기 위해 모델을 다운받고, 인퍼런스를 실행합니다. 

model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.68it/s]


In [27]:
print(f"PyTorch version       : {torch.__version__}")
print(f"Transformers version  : {transformers.__version__}")
print(f"TRL version           : {trl.__version__}")
print(f"CUDA available        : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version      : {torch.version.cuda}")

PyTorch version       : 2.6.0+cu124
Transformers version  : 4.50.2
TRL version           : 0.15.2
CUDA available        : True
CUDA version      : 12.4


In [28]:
login(
  token=os.getenv("Huggingface_Token"),
)

In [29]:
dataset = load_dataset("daje/kotext-to-sql-v1")
dataset     

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'response', 'source', 'text', 'ko_instruction'],
        num_rows: 262208
    })
})

In [30]:
def add_length_column(dataset):
    df = dataset.to_pandas()
    df["total_length"] = 0
    for column_name in ["instruction", "input", "response"]:
        num_words = df[column_name].astype(str).str.split().apply(len)
        df["total_length"] += num_words

    return df

df = add_length_column(dataset["train"])

def filter_by_total_length(df, difficulty, number_of_samples, random_state=8888):  # random_state 추가
    if difficulty == "easy":
        return df[df["total_length"].between(10, 100)].sample(n=number_of_samples, random_state=random_state)  # iloc 대신 sample 사용
    elif difficulty == "moderate":
        return df[df["total_length"].between(101, 300)].sample(n=number_of_samples, random_state=random_state)
    elif difficulty == "difficult":
        return df[df["total_length"].between(301, 1000)].sample(n=number_of_samples, random_state=random_state)

print(max(df["total_length"].to_list()), min(df["total_length"].to_list()))

923 14


In [31]:
# trl docs에 보면 이와 같은 방식으로 SFT Trainer용 데이터를 만들 수 있습니다.
# docs에서는 eos_token을 별도로 추가하라는 안내는 없지만, 저자는 습관적으로 eos_token을 붙혀줍니다.
def get_chat_format(element):
    system_prompt = (
        "You are a helpful programmer assistant that excels at SQL. "
        "Below are sql tables schemas paired with instruction that describes a task. "
        "Using valid SQLite, write a response that appropriately completes the request for the provided tables."
    )
    user_prompt = "### instruction:{instruction} ### Input:{input} ### response:"
    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt.format_map(element)},
            {"role": "assistant", "content": element["response"]+tokenizer.eos_token},
        ]
    }

tokenizer.padding_side = 'right'                      

# 데이터를 일괄적으로 대화형식으로 변경합니다.
def tokenize_messages(element):
    """
    2) apply_chat_template + tokenizer(...)를 통해
       input_ids와 attention_mask를 만들어 반환합니다.
    """
    # 위 단계에서 "messages"가 이미 Dataset에 들어가있다고 가정
    formatted = tokenizer.apply_chat_template(
        element["messages"],  # messages 리스트
        tokenize=False
    )
    outputs = tokenizer(formatted)
    return {
        "input_ids": outputs["input_ids"],
        "attention_mask": outputs["attention_mask"]
    }

easy = filter_by_total_length(df, "easy", 100)
medium = filter_by_total_length(df, "moderate", 100)
hard = filter_by_total_length(df, "difficult", 100)
dataset = pd.concat([easy, medium, hard])
dataset = dataset.sample(frac=1, random_state=8888)  # random_state 추가
dataset = Dataset.from_pandas(dataset)
easy.shape, medium.shape, hard.shape, dataset.shape

temp_dataset = dataset.map(get_chat_format, remove_columns=dataset.features, batched=False)

# train과 test 데이터를 0.9와 0.1로 분할합니다.
temp_dataset = temp_dataset.train_test_split(test_size=0.05, seed=42)

temp = tokenizer.apply_chat_template(temp_dataset["train"][0]["messages"], tokenizer=False)
print(tokenizer.decode(temp))

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map: 100%|██████████| 300/300 [00:00<00:00, 14686.97 examples/s]

<|im_start|>system
You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.<|im_end|>
<|im_start|>user
### instruction:what is the total number of patients discharged from the hospital in this year? ### Input:CREATE TABLE icustays (
    row_id number,
    subject_id number,
    hadm_id number,
    icustay_id number,
    first_careunit text,
    last_careunit text,
    first_wardid number,
    last_wardid number,
    intime time,
    outtime time
)

CREATE TABLE d_icd_procedures (
    row_id number,
    icd9_code text,
    short_title text,
    long_title text
)

CREATE TABLE diagnoses_icd (
    row_id number,
    subject_id number,
    hadm_id number,
    icd9_code text,
    charttime time
)

CREATE TABLE chartevents (
    row_id number,
    subject_id number,
    hadm_id number,
    icustay_id number,
    i

In [32]:
# 일부 데이터만 샘플링하고 싶은 경우 
# easy = filter_by_total_length(df, "easy", 10000)
# medium = filter_by_total_length(df, "moderate", 10000)
# hard = filter_by_total_length(df, "difficult", 2000)
# dataset = pd.concat([easy, medium, hard])
# dataset = dataset.sample(frac=1, random_state=8888)  # random_state 추가
# dataset = Dataset.from_pandas(dataset)
# easy.shape, medium.shape, hard.shape, dataset.shape

# 전체 데이터로 학습을 할 경우 
dataset = Dataset.from_pandas(df)

In [33]:
# train과 test 데이터를 0.9와 0.1로 분할합니다.
dataset = dataset.map(
    get_chat_format,
    batched=False,
    remove_columns=dataset.features,  # 원하시면 제거
)
dataset = dataset.train_test_split(test_size=0.02, seed=42)
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

dataset = dataset.map(
    tokenize_messages,
    batched=False,
    remove_columns=["messages"],  # 이제 messages를 더이상 쓰지 않는다면 제거
)

Map: 100%|██████████| 5245/5245 [00:03<00:00, 1378.86 examples/s]


In [34]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 256963
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 5245
    })
})

In [35]:
from trl import DataCollatorForCompletionOnlyLM
response_template = "<|im_start|>assistant\n" # 만약 실제로 줄바꿈까지 포함되어 있다면 \n까지 넣어줘야 합니다.
# response_llama_template = "<|start_header_id|>assistant<|end_header_id|>\n"

response_template_ids = tokenizer.encode(
    response_template, 
    add_special_tokens=False
    )
collator = DataCollatorForCompletionOnlyLM(
    response_template_ids,
    tokenizer=tokenizer
)

In [37]:
peft_config = LoraConfig(
        lora_alpha=128,                            
        lora_dropout=0.05,                         # Lora 학습 때 사용할 dropout 확률을 지정합니다. 드롭아웃 확률은 과적합 방지를 위해 학습 중 무작위로 일부 뉴런을 비활성화하는 비율을 지정합니다.
        r=256,                                     # Lora의 저차원 공간의 랭크를 지정합니다. 랭크가 높을수록 모델의 표현력이 증가하지만, 계산 비용도 증가합니다.
        bias="none",                               # Lora 적용 시 바이어스를 사용할지 여부를 설정합니다. "none"으로 설정하면 바이어스를 사용하지 않습니다.
        target_modules=["q_proj", "o_proj",        # Lora를 적용할 모델의 모듈 리스트입니다.
                        "k_proj", "v_proj"
                        "up_proj", "down_proj",
                        "gate_proj",
                        ],
        task_type="CAUSAL_LM",                     # 미세 조정 작업 유형을 CAUSAL_LM으로 지정하여 언어 모델링 작업을 수행합니다.
)


args = TrainingArguments(
    output_dir="Qwen2.5-260000_en",         # 모델 저장 및 허브 업로드를 위한 디렉토리 지정 합니다.
    num_train_epochs=1,                     # number of training epochs
    # max_steps=100,                        # 100스텝 동안 훈련 수행합니다.
    per_device_train_batch_size=1,          # 배치 사이즈 설정 합니다.
    gradient_accumulation_steps=2,          # 4스텝마다 역전파 및 가중치 업데이트합니다.
    gradient_checkpointing=False,           # 메모리 절약을 위해 그래디언트 체크포인팅 사용합니다.
    optim="adamw_torch_fused",              # 메모리 효율화할 수 있는 fused AdamW 옵티마이저 사용합니다.
    logging_steps=5000,                     # 10스텝마다 로그 기록합니다.
    save_strategy="steps",                  # 매 에폭마다 체크포인트 저장합니다.
    learning_rate=5e-5,                     # 학습률 2e-4로 설정 (QLoRA 논문 기반)합니다.
    bf16=True,                              # 정밀도 설정으로 학습 속도 향상합니다.
    tf32=True,
    max_grad_norm=0.3,                      # 그래디언트 클리핑 값 0.3으로 설정합니다.
    warmup_ratio=0.03,                      # 워밍업 비율 0.03으로 설정 (QLoRA 논문 기반)합니다.
    lr_scheduler_type="constant",           # 일정한 학습률 스케줄러 사용합니다.
    push_to_hub=False,                      # 훈련된 모델을 Hugging Face Hub에 업로드합니다.
    report_to="none",                       # 관찰하고 싶다면, wandb나 tensorboard 등을 넣으면 됩니다. 
)


trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    tokenizer=tokenizer,
    data_collator=collator,
)


Truncating train dataset: 100%|██████████| 256963/256963 [01:19<00:00, 3247.18 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
# trainer를 학습합니다.
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: daje0601. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss


In [19]:
# 메모리 초기화
del model
del trainer
torch.cuda.empty_cache()

In [19]:
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

# CUDA_VISIBLE_DEVICES=3 vllm serve "Qwen/Qwen2.5-Coder-7B-Instruct" --tensor-parallel-size 1 --host 0.0.0.0 --port 8000 --gpu-memory-utilization 0.90 --enable-lora --max-lora-rank 64 --lora-modules sql-lora="lora 경로" 
# CUDA_VISIBLE_DEVICES=3 vllm serve "daje/Qwen2.5-coder-7B-en-all-merged" --tensor-parallel-size 1 --host 0.0.0.0 --port 8000 --gpu-memory-utilization 0.90 

In [18]:
from datasets import load_dataset
eval_dataset = load_dataset("json", data_files="./test_dataset.json", split="train")
eval_dataset

Generating train split: 5245 examples [00:00, 184958.29 examples/s]


Dataset({
    features: ['messages'],
    num_rows: 5245
})

In [22]:
eval_dataset[0]["messages"][:2]

[{'content': 'You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.',
  'role': 'system'},
 {'content': '### instruction:calculate the maximum days for which patients who died before 2155 stayed in hospital. ### Input:CREATE TABLE diagnoses (\n    subject_id text,\n    hadm_id text,\n    icd9_code text,\n    short_title text,\n    long_title text\n)\n\nCREATE TABLE lab (\n    subject_id text,\n    hadm_id text,\n    itemid text,\n    charttime text,\n    flag text,\n    value_unit text,\n    label text,\n    fluid text\n)\n\nCREATE TABLE demographic (\n    subject_id text,\n    hadm_id text,\n    name text,\n    marital_status text,\n    age text,\n    dob text,\n    gender text,\n    language text,\n    religion text,\n    admission_type text,\n    days_stay text,\n    insurance text,\n    ethnicity text

In [22]:
idx = 0
sql_chat_completion = client.chat.completions.create(
    model="lora_adapter1",
    messages=eval_dataset[idx]["messages"][:2],
    temperature=0.0,
    max_tokens=500,
)


# merged를 한경우 
# idx = 0
# sql_chat_completion = client.chat.completions.create(
#     model="daje/Qwen2.5-coder-7B-en-all-merged",
#     messages=eval_dataset[idx]["messages"][:2],
#     temperature=0.0,
#     max_tokens=500,
# )

In [26]:
eval_dataset[idx]["messages"][2]

{'content': 'SELECT MAX(demographic.days_stay) FROM demographic WHERE demographic.dod_year < "2155.0"<|im_end|>',
 'role': 'assistant'}

In [6]:
print(sql_chat_completion.choices[0].message.content)

SELECT MAX(demographic.days_stay) FROM demographic WHERE demographic.dod_year < "2155.0"


In [9]:
# 이런 경우 때문에 OpenAI 채점이 필요합니다. 
eval_dataset[idx]["messages"][2]["content"].replace("<|im_end|>", "").strip() == sql_chat_completion.choices[0].message.content.strip()

True

In [10]:
from tqdm.auto import tqdm 
result = [] 
for idx in tqdm(range(len(eval_dataset))):
    sql_chat_completion = client.chat.completions.create(
    model="lora_adapter1",
    messages=eval_dataset[idx]["messages"][:2],
    temperature=0.1,
    max_tokens=500,
    )
    result.append(
        (
            eval_dataset[idx]["messages"][:2],
            eval_dataset[idx]["messages"][2]["content"], 
            sql_chat_completion.choices[0].message.content
        )
    )

100%|██████████| 5245/5245 [46:23<00:00,  1.88it/s]  


In [ ]:
# 실수로 다시 실행해도 파일이 사라지지 않도록 하기 위해 timestamp를 사용하는걸 습관화하시는게 좋습니다. 
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"./inference_output_qwen2.5_en_{timestamp}.json"
 # 저장할 파일 이름
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

print(f"결과가 {output_file} 파일에 저장되었습니다.")

In [1]:
import json 
output_file = "/root/workspace/text_to_sql/inference_output_qwen2.5_en_20250112_120251.json"

with open(output_file, "r") as file:
    result = json.load(file)
result[0]

[[{'content': 'You are a helpful programmer assistant that excels at SQL. Below are sql tables schemas paired with instruction that describes a task. Using valid SQLite, write a response that appropriately completes the request for the provided tables.',
   'role': 'system'},
  {'content': '### instruction:calculate the maximum days for which patients who died before 2155 stayed in hospital. ### Input:CREATE TABLE diagnoses (\n    subject_id text,\n    hadm_id text,\n    icd9_code text,\n    short_title text,\n    long_title text\n)\n\nCREATE TABLE lab (\n    subject_id text,\n    hadm_id text,\n    itemid text,\n    charttime text,\n    flag text,\n    value_unit text,\n    label text,\n    fluid text\n)\n\nCREATE TABLE demographic (\n    subject_id text,\n    hadm_id text,\n    name text,\n    marital_status text,\n    age text,\n    dob text,\n    gender text,\n    language text,\n    religion text,\n    admission_type text,\n    days_stay text,\n    insurance text,\n    ethnicity t

In [2]:
# ------------------------------------------------------------------------------
# 1. 1차 필터: Exact Match
#    (문자열이 완전히 동일하면 True, 아니면 False)
# ------------------------------------------------------------------------------
generated_result = [temp[1].replace("<|im_end|>", "").strip() == temp[2].replace("<|im_end|>", "").strip() for temp in result]

# Exact Match 기준으로 ACC(정확도)를 구합니다.
accuracy = sum(generated_result)/len(generated_result)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 76.47%


In [ ]:
import os
import json
from pathlib import Path
from pydantic import BaseModel
from openai import OpenAI
from pqdm.processes import pqdm


# ------------------------------------------------------------------------------
# 2. False(= 서로 다른 쿼리)인 것만 모아서 OpenAI에 의미적 비교
# ------------------------------------------------------------------------------
differing_items = []  # (index, problem, generated_sql, ground_truth_sql)
for i, is_match in enumerate(generated_result):
    if not is_match:
        problem_desc, gen_sql, gt_sql = result[i]
        differing_items.append((i, problem_desc, gen_sql, gt_sql))

print(f"[2차 OpenAI 검증] Exact Match로는 {len(differing_items)}개가 불일치로 간주되어, OpenAI로 의미 비교를 진행합니다.\n")


[2차 OpenAI 검증] Exact Match로는 1234개가 불일치로 간주되어, OpenAI로 의미 비교를 진행합니다.

In [ ]:
# ------------------------------------------------------------------------------
# OpenAI 및 Pydantic 설정
# ------------------------------------------------------------------------------

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

# 결과 파일 저장 폴더
save_folder = Path("./openai_result_qwen2.5_all-en")
save_folder.mkdir(parents=True, exist_ok=True)

class SqlComparisonResult(BaseModel):
    # answer: 1 (의미적으로 동일) 또는 0 (의미적으로 다름)
    answer: int
    explanation: str


def one_compare_sql_semantics(item_tuple) -> dict:
    """
    item_tuple: (idx, problem_desc, gen_sql, gt_sql)
    OpenAI에 의미적 비교 요청 -> 'answer'(1/0), 'explanation' 반환
    """
    idx, problem_description, generated_query, ground_truth_query = item_tuple
    save_path = save_folder / f"result_{idx}.json"

    # 이미 결과 파일이 존재하면, 재요청하지 않고 스킵
    if save_path.exists():
        print(f"[{idx}] 이미 파일이 있습니다.")
        # 저장된 결과 불러오기
        return None

    # 프롬프트 작성
    prompt = """아래에는 한 개의 문제와 두 개의 SQL 쿼리가 주어집니다.

이때, "두 쿼리가 문자열로 달라도 실제로 동일한 결과를 반환하는지"를 판단하세요.  
구체적으로 다음 기준에 따라 판정하십시오:

1. 결과 집합(ROW 세트)이 동일한지 여부만을 판단하십시오.
- 대소문자 차이, 공백, 세미콜론 유무, 쿼리 포맷(줄바꿈 등), 작은따옴표 vs 큰따옴표, 
    테이블/컬럼 별칭 등이 다르더라도 결과 집합이 같다면 '동일'이라고 간주합니다.
2. 문법상 다른 함수나 구조를 사용했더라도 결과가 같으면 동일합니다.
- 예: GROUP BY 대상이 달라도 실제 결과가 동일하면 '동일'
        ORDER BY id vs ORDER BY season 이 실제로 같은 순서를 의미하면 '동일'
3. 만약 실제 결과가 다르다면, answer에 "0"을 적고 그 이유를 설명해 주세요.
4. 답변은 아래 JSON 형식으로만 작성해 주세요 (다른 텍스트, 문장 포함 금지):
- **"answer"**: 
  - "1"이면 두 쿼리가 의미적으로 동일한 결과를 반환한다는 의미
  - "0"이면 의미적으로 다른 결과를 반환한다는 의미  
- **"explanation"**: 
  - 왜 동일한지 또는 왜 다른지를 한글로 간단히 설명  
  - 쿼리가 완전히 같은지, 컬럼명이 달라도 동일한 컬럼을 참조하고 있는지, GROUP BY 차이가 결과에 영향을 주는지/주지 않는지, etc. 상세히 기술

### (예시) 
1. **(동일한 결과 예시)**  
   - 쿼리 A: `SELECT "Semifinalists" FROM table_31066 WHERE "Runner-up" = 'Sammy Giammalva'`  
   - 쿼리 B: `SELECT "Semifinalists" FROM table_31066 WHERE "Runner-up" = 'sammy zimmalva'`  
   - 이 경우는 **Runner-up 이름이 전혀 다르므로 실제 결과가 다르다** → `answer: 0`  
   
2. **(동일한 결과 예시)**  
   - 쿼리 A: `SELECT T1.CName FROM COURSE AS T1 JOIN ENROLLED_IN AS T2 ON T1.CID = T2.CID GROUP BY T2.CID HAVING COUNT(*) >= 5`  
   - 쿼리 B: `SELECT T1.CName FROM COURSE T1 JOIN ENROLLED_IN T2 ON T1.CID = T2.CID GROUP BY T1.CName HAVING COUNT(*) >= 5`  
   - 만약 `CName`과 `CID`가 1:1 대응되어 있어 실제 반환되는 행이 **동일**하다면 `answer=1`.  
   - 만약 CName에 중복이 있을 수 있어 결과가 달라진다면 `answer=0`.

3. **(동일한 결과 예시)**  
   - 쿼리 A: `SELECT "west" FROM table_204_1 ORDER BY "season" DESC LIMIT 1`  
   - 쿼리 B: `SELECT "west" FROM table_204_1 ORDER BY id DESC LIMIT 1`  
   - 만약 `id`와 `"season"`이 완전히 같은 순으로 증가한다면 **결과가 같다** → `answer=1`.  
   - 만약 그렇지 않다면(예: id가 엉뚱하게 매겨짐) → `answer=0`.

위 사항을 참고하여, 
- **두 SQL 쿼리가 의미적으로 동일한지** (결과 집합이 같은지)  
- 대소문자, 공백, 별칭 차이 등은 **결과에 영향을 주지 않으므로** `answer=1`로 처리  
- 실제 결과가 다른 경우만 `answer=0` 처리하고 왜 다른지 짧게 설명  

이 지침에 **엄격히** 따라 판단하십시오.

    {{
    "answer": "1 또는 0",
    "explanation": "이유를 한글로 간결히"
    }}

    위 지침을 만족하도록 문제와 쿼리 정보를 참고해 판단해 주세요.

    ----------------------------
    문제 설명:
    {problem_description}

    생성된 쿼리:
    {generated_query}

    정답(원본) 쿼리:
    {ground_truth_query}

    판단 후, 
    - 두 쿼리가 의미적으로 동일하면 "answer": "1"
    - 다르면 "answer": "0"
    그리고 "explanation"에 그 이유를 한글로 적어주세요.
    """.strip()

    try:
        # Structured Output
        completion = client.beta.chat.completions.parse(
            model="gpt-4o-mini",  # 사용 가능한 모델로 변경
            messages=[
                {
                    "role": "system",
                    "content": (
                        "주어진 문제의 맥락에서 SQL 쿼리의 의미적 의미를 비교하는 유용한 도우미입니다. "
                        "explanation은 한글로 작성하세요."
                    )
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            response_format=SqlComparisonResult,
        )
        parsed_result = completion.choices[0].message.parsed
        
        # dict로 변환
        result_dict = {
            "index": idx,
            "answer": parsed_result.answer,
            "explanation": parsed_result.explanation
        }
        # 파일로 저장
        with open(save_path, "w", encoding="utf-8") as f:
            json.dump(result_dict, f, ensure_ascii=False, indent=4)

        # print(f"[{idx}] Saved: answer={parsed_result.answer}")
        return result_dict

    except Exception as e:
        print(f"[{idx}] Error processing: {e}")
        return {
            "index": idx,
            "answer": -1,
            "explanation": f"Error: {str(e)}"
        }


In [5]:

# ------------------------------------------------------------------------------
# 3. pqdm을 이용한 병렬 처리
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    from pqdm.processes import pqdm

    # 병렬 처리 (CPU core 개수에 맞춰 n_jobs 조정)
    results = pqdm(differing_items, one_compare_sql_semantics, n_jobs=5)

    # 2차 검증 결과 중 answer가 1인 것(의미적으로 동일하다고 판정)
    # => 몇 개나 되는지 집계
    valid_results = [r for r in results if r["answer"] in (0,1)]
    if len(valid_results) == 0:
        print("\n[OpenAI 검증] 유효 결과가 없습니다.")
    else:
        semantically_same = sum(r["answer"] == 1 for r in valid_results)
        semantically_diff = sum(r["answer"] == 0 for r in valid_results)
        print(f"\n[OpenAI 검증] 의미적으로 동일(1): {semantically_same}, "
              f"다름(0): {semantically_diff}, 총 {len(valid_results)}개")

QUEUEING TASKS | : 100%|██████████| 1234/1234 [00:00<00:00, 15861.13it/s]
PROCESSING TASKS | : 100%|██████████| 1234/1234 [04:30<00:00,  4.56it/s]
COLLECTING RESULTS | : 100%|██████████| 1234/1234 [00:00<00:00, 206543.40it/s]


[OpenAI 검증] 의미적으로 동일(1): 342, 다름(0): 892, 총 1234개


In [6]:
import os
import json
from pathlib import Path

# 결과가 저장된 폴더
save_folder = Path("./openai_result_qwen2.5_all-en")

# 파일 목록 가져오기
json_files = sorted(save_folder.glob("result_*.json"))

num_files = 0
num_correct = 0

for file_path in json_files:
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
        # data 형태 예시:
        # {
        #   "answer": 1,
        #   "explanation": "..."
        # }
        
        # answer가 1이면 "의미적으로 동일"이라고 판단한 것이므로 맞았다고 봄
        if data.get("answer") == 1:
            num_correct += 1
        num_files += 1

# 간단한 통계
if num_files > 0:
    accuracy = (num_correct / num_files) * 100
    print(f"정답 개수: {num_correct}/{num_files} 개 (정확도: {accuracy:.2f}%)")
else:
    print("결과 파일이 없습니다.")


정답 개수: 342/1234 개 (정확도: 27.71%)


In [7]:
total_accuracy = (sum(generated_result) + num_correct) / len(generated_result) * 100
print(f"정확도 : {total_accuracy:.2f}%")

정확도 : 82.99%
